# I. Context and Description 

- Background Information
    
    The complete dataset contains 720,000 competitive matches from the popular game PlayerUnknown's Battlegrounds (PUBG). PUBG is a first/third-person shooter battle royale style game developed and published by PUBG Corporation. The game is based on previous mods that were created by Brendan "PlayerUnknown" Greene for other games using the film Battle Royale for inspiration, and expanded into a standalone game under Greene's creative direction. 
    
    In the game, up to 100 players parachute onto a remote island where teams and players fight to death until only one remains. Players are dropped from an airplane onto the island where they are to scavenge towns and buildings for weapons, vehicles and supplies. Players will then decide to either fight or hide with the ultimate goal of being the last one standing. The battleground shrinks a few minutes into the game as a bluezone appears and deals damage to anyone that stands inside the bluezone while sparing whoever is within the safe zone. As the available safe area of the game's map decreases in size over time, surviving players are directed into tighter areas and forced encouters. The last player or team standing wins the round.
    
    For analysis purposes and due to technical difficulties, we are only using one-fifth observations of the complete dataset. 

- Data Collection

    The data was collected by a Kaggle user with username "KP" in January 2018. He used a scraping methodology that first starts with an initial seed player, which he chose to be his own account (a somewhat low rank individual). Then, using the seed player, he scrapes for all players that it has encountered in its historical matches. Taking a random subset of 5000 players from this and scraping for their historical games, we obtain our final dataset. 
    
    A potential problem is that this could produce an unrepresentative sample of all games played since it is likely that players are queued and matched with other players of similar rank levels. In our case, it could be that the dataset is more representative of lower-tier gameplay. However, given the simplicity of the dataset, this should not be a significant issue. 

# II. Exploratory Analysis

# III. Data Modeling